In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false" 
#Change the path of some HF environment variables to store the download data (model and dataset) from the hub to a choosen location
os.environ['HF_HOME'] = "../.cache"
os.environ['HF_HUB_CACHE'] = "../.cache"
os.environ['HF_DATASETS_CACHE'] = "../.cache"
from transformers import PaliGemmaProcessor, AutoTokenizer, DonutImageProcessor
from modeling_divedoc import DIVEdoc
from processing_divedoc import PaliGemmaProcessor
from datasets import load_dataset

/home/rbencharef/miniconda3/envs/divedoc-plateform-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
hf_token = os.getenv("HF_TOKEN")

In [2]:
model = DIVEdoc.from_pretrained(
    "JayRay5/DIVE-Doc-FRD",
    trust_remote_code=True)

Vision config in end-to-end model: swinpam
[64, 64]
[64, 64]


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 17.10it/s]


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/paligemma-3b-ft-docvqa-896",token=jf_token)
image_processor = DonutImageProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
image_processor.image_seq_length = 4096
image_processor.size["height"],image_processor.size["width"] = model.config.vision_config.encoder_config.image_size[0],model.config.vision_config.encoder_config.image_size[1]

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/paligemma-3b-ft-docvqa-896.
401 Client Error. (Request ID: Root=1-69431d50-7a420b800870dc8120c74dca;66ef4cb4-acc2-4d50-bebe-c69289beb4c7)

Cannot access gated repo for url https://huggingface.co/google/paligemma-3b-ft-docvqa-896/resolve/main/config.json.
Access to model google/paligemma-3b-ft-docvqa-896 is restricted. You must have access to it and be authenticated to access it. Please log in.

In [4]:
processor = PaliGemmaProcessor(tokenizer=tokenizer, image_processor=image_processor)

In [5]:
test_dataset = load_dataset("pixparse/docvqa-single-page-questions", split="test",streaming=True)
batch_dataset = test_dataset.batch(2) 
iterative_dataset = iter(batch_dataset)
batch = next(iterative_dataset)

In [18]:
#preprocessing 
imgs = []
for i in batch["image"]:
    imgs.append(i.convert('RGB'))
txt = batch["question"]
model_inputs = processor(text=txt, images=imgs, return_tensors="pt",padding=True)
input_len = model_inputs["input_ids"].shape[-1]
    

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


In [17]:
batch["question"]

['Routing#?', 'Heading of the page?']

In [12]:
model_inputs["input_ids"].shape

torch.Size([1, 4109])

In [28]:
processor.tokenizer.decode([0,5,6,8], skip_special_tokens=True)

'<2mass>[@BOS@]<unused1>'

In [7]:
imgs = []
for i in batch["image"]:
    imgs.append(i.convert('RGB'))
processor.image_processor(images=imgs[0])

{'pixel_values': [array([[[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]],

       [[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]],

       [[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]]],
      shape=(3, 2560, 1920), dtype=float32)]}

In [19]:
model.config

DIVEdocConfig {
  "_attn_implementation_autoset": true,
  "_vocab_size": 257152,
  "architectures": [
    "DIVEdoc"
  ],
  "auto_map": {
    "AutoConfig": "JayRay5/DIVE-Doc-FRD--configuration_divedoc.DIVEdocConfig",
    "AutoModelForCausalLM": "JayRay5/DIVE-Doc-FRD--modeling_divedoc.DIVEdoc"
  },
  "bos_token_id": 2,
  "eos_token_id": 1,
  "hidden_size": 2048,
  "image_token_index": 257152,
  "model_type": "DIVEdoc",
  "pad_token_id": 0,
  "projection_dim": 2048,
  "text_config": {
    "attention_bias": false,
    "attention_dropout": 0.0,
    "head_dim": 256,
    "hidden_act": "gelu_pytorch_tanh",
    "hidden_activation": null,
    "hidden_size": 2048,
    "initializer_range": 0.02,
    "intermediate_size": 16384,
    "max_position_embeddings": 8192,
    "model_type": "gemma",
    "num_attention_heads": 8,
    "num_hidden_layers": 18,
    "num_image_tokens": 4096,
    "num_key_value_heads": 1,
    "rms_norm_eps": 1e-06,
    "rope_theta": 10000.0,
    "torch_dtype": "float32",
    "use